In [6]:
# Function for variables
def variables_decl(month_par, year_par):
    #Packages
    from datetime import timedelta
    from datetime import datetime
    # Variable declaration
    a = timedelta(seconds=300)
    month = str(month_par)      # Cast to string in order to use it in image titles (does not accept int types)
    year = str(year_par)    # Cast to string in order to use it in image titles (does not accept int types)

    return (a, month,year)

In [7]:
# Function for database import
def db_import():
    #Packages
    import cx_Oracle
    import pandas as pd
    
    # Initializing database connection
    con = cx_Oracle.connect('STC_SETRAG/art@localhost/XE')
    cur = con.cursor()

    # First Query
    query=cur.execute('select * from historico_mensagens')
    df = pd.DataFrame(query)
    # Second Query
    query=cur.execute('select EV_NOM_MAC from ELEM_VIA where TE_ID_TP = 3')
    df_sb = pd.DataFrame(query)

    cur.close()
    con.close()
    
    return (df,df_sb)

In [8]:
# Function for formatting dataframe
def df_format(df, df_sb):
    # Adding names to the columns of the dataframes
    df.columns=['HM_ID_HM','HM_LOCO_MSG','HM_PRF_TRM','HM_OBC_MSG','HM_NUM_MSG','HM_TP_MSG','HM_DAT_OBC','HM_DAT_STC',
                'HM_TXT_MSG','HM_TXT_TW','HM_LAT_HM','HM_LON_HM','HM_ENV_RCB','HM_MSG_VOZ','HM_NUM_FUN','HM_ID_AUX','HM_SIT_ENV','MT_ID_MT']
    #df_sb.columns=['NAME']
    # Dropping tables with no use
    df = df.drop(['HM_TP_MSG','HM_TXT_TW','MT_ID_MT','HM_ID_AUX','HM_OBC_MSG','HM_LAT_HM','HM_LON_HM'], axis=1)
   
   # for i in range(0,len(df_sb['NAME']),1):
   #     df_sb['SECTION_DEBUT'] = "Section Début: " + df_sb.loc[:,'NAME'] + " "
    #    df_sb['CANTON'] = "Canton: " + df_sb.loc[:,'NAME'] + " "
    #    df_sb['SECTION_TETE'] = "Section da Tête: " + df_sb.loc[:,'NAME'] + " "

    df_sb['COUNTS_SENT'] = 0
    df_sb['COUNTS_RECEIVED'] = 0
    df_sb['COUNTS_TOTAL'] = 0
    df_sb['TOTAL_TIME_SENT'] = 0.0
    df_sb['TOTAL_TIME_RECEIVED'] = 0.0
    df_sb['TOTAL_TIME'] = 0.0
    df_sb['AVERAGE_TIME_SENT'] = 0.0
    df_sb['AVERAGE_TIME_RECEIVED'] = 0.0
    df_sb['AVERAGE_TIME'] = 0.0
    df_sb['%_SENT'] = 0
    df_sb['%_RECEIVED'] = 0
    df_sb['%_TOTAL'] = 0
    
    return(df, df_sb)

In [9]:
# Function for splitting dataframe in 'Sent' and 'Received' messages
def df_split(df):
    #Packages
    import pandas as pd
    
    df_sent = df.loc[df['HM_ENV_RCB']=='E']
    df_received = df.loc[df['HM_ENV_RCB']=='R']

    # Removing the original dataframe
    df = pd.DataFrame(None)
    
    return(df, df_sent, df_received)

In [1]:
# Function for filtering (part I)
def filter_I (df_sent,df_received,month,year):
    # Sent DataFrame
    # 1) Filter messages from january 2021
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']>= year + '-' + month +'-01']
    if(month == '1' or month == '3' or month == '5' or month == '7' or month == '8' or month == '10' or month == '12'):
        df_sent = df_sent.loc[df_sent['HM_DAT_STC']<= year + '-' + month +'-31']
    elif(month == '4' or month == '6' or month == '9' or month == '11'):
        df_sent = df_sent.loc[df_sent['HM_DAT_STC']<= year + '-' + month +'-30']
    else:
        df_sent = df_sent.loc[df_sent['HM_DAT_STC']<= year + '-' + month +'-28']  # February
    # 2) Filter all messages that contain OBC time
    df_sent = df_sent.loc[df_sent['HM_DAT_OBC'].notnull()]

    # Received DataFrame
    # 1) Filter all messages that did not expired
    df_received = df_received.loc[df_received['HM_SIT_ENV']=='E']
    # 2) Filter messages from january 2021
    df_received = df_received.loc[df_received['HM_DAT_STC']>= year + '-' + month +'-01']
    if(month == '1' or month == '3' or month == '5' or month == '7' or month == '8' or month == '10' or month == '12'):
        df_received = df_received.loc[df_received['HM_DAT_STC']<= year + '-' + month +'-31']
    elif(month == '4' or month == '6' or month == '9' or month == '11'):
        df_received = df_received.loc[df_received['HM_DAT_STC']<= year + '-' + month +'-30']
    else:
        df_received = df_received.loc[df_received['HM_DAT_STC']<= year + '-' + month +'-28']  # February
    # 3) Filter all messages that contain OBC time
    df_received = df_received.loc[df_received['HM_DAT_OBC'].notnull()]
    # 4) Filter all messages that have a date later than needed (avoiding future date messages)
    month 
    df_received = df_received.loc[df_received['HM_DAT_OBC']< year + '-' + str(int(month)+1) +'-02']
    
    
    return(df_sent, df_received)

In [2]:
# Function for returning the total amount of sent messages, received messages and total (sent + received) messages
def count_messages(df_sent, df_received):
    # Sent
    totalSentMessages = len(df_sent['HM_ID_HM'])
    # Received
    totalReceivedMessages = len(df_received['HM_ID_HM'])
    # Total
    totalMessages = totalSentMessages + totalReceivedMessages

    return(totalSentMessages, totalReceivedMessages, totalMessages)

In [3]:
# Function for returning the average delivery time of sent, received and total (sent + received) messages
def average_delivery(df_sent,df_received, totalSentMessages, totalReceivedMessages):
    #Packages
    from datetime import timedelta
    from datetime import datetime   
    import numpy as np
    
    # Sent
    df_sent['DELIVERY_TIME'] = df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']
    # Converting to string in order to execute a mathematic operation
    df_sent['DELIVERY_TIME'] = df_sent['DELIVERY_TIME']/np.timedelta64(1, 's')
    totalSent = sum(df_sent.iloc[:,11])
    averageSentDelivery = round(totalSent/totalSentMessages,0)
    # Converting from float to mm:ss
    averageSentDelivery = timedelta(seconds=averageSentDelivery)
    averageSentDelivery=str(averageSentDelivery)

    # Received
    df_received['DELIVERY_TIME'] = df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']
    # Converting to string in order to execute a mathematic operation
    df_received['DELIVERY_TIME'] = df_received['DELIVERY_TIME']/np.timedelta64(1, 's')
    totalReceived = sum(df_received.iloc[:,11])
    averageReceivedDelivery = round(totalReceived/totalReceivedMessages,0)
    # Converting from float to mm:ss
    averageReceivedDelivery = timedelta(seconds=averageReceivedDelivery)
    averageReceivedDelivery = str(averageReceivedDelivery)
    
    # Return the average delivery time of Sent and Received messages combined
    averageDelivery = round((totalSent + totalReceived)/(totalSentMessages + totalReceivedMessages),0)
    # Converting from float to mm:ss
    averageDelivery = timedelta(seconds=averageDelivery)
    averageDelivery = str(averageDelivery)


    return(averageSentDelivery, averageReceivedDelivery, averageDelivery)

In [1]:
# Function to create Density graph
def dens_graph(df_sent, df_received, month, year):
    #Packages
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    sns.distplot(df_sent['DELIVERY_TIME'],hist=False,kde=True,color = 'cornflowerblue', label = 'Sent')
    sns.distplot(df_received['DELIVERY_TIME'],hist=False,kde=True, color = 'tab:red', label = 'Received')
    plt.title('Delivery times '+month+'-'+year)
    plt.xlabel("Delivery Times (in seconds)") 
    plt.legend(loc='best')
    plt.xlim(0, 500)
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/density'+'_'+ month + '_' + year +'.png')
    plt.show()

In [2]:
# Function for returning % and total amount of messages that took more than 5min for both sent, received and total(sent + received) messages.
def calc_delay(df_sent, df_received, totalSentMessages, totalReceivedMessages, a):
    # Sent
    totalSentDelay= df_sent.loc[(df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']) >= a]
    totalSentDelay = len(totalSentDelay)
    percentageSentDelay=(totalSentDelay/totalSentMessages)*100
    percentageSentDelay = round(percentageSentDelay,2)
    percentageSentDelay = str(percentageSentDelay)+ "%"

    # Received
    totalReceivedDelay= df_received.loc[(df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']) >= a]
    totalReceivedDelay = len(totalReceivedDelay)
    percentageReceivedDelay=(totalReceivedDelay/totalReceivedMessages)*100
    percentageReceivedDelay = round(percentageReceivedDelay,2)
    percentageReceivedDelay = str(percentageReceivedDelay)+ "%"

    # Total
    totalDelay = totalSentDelay + totalReceivedDelay
    totalMessages = totalSentMessages + totalReceivedMessages
    percentageTotalDelay = (totalDelay/totalMessages)*100
    percentageTotalDelay = round(percentageTotalDelay,2)
    percentageTotalDelay = str(percentageTotalDelay)+"%"


    return(percentageSentDelay, totalSentDelay, percentageReceivedDelay, totalReceivedDelay, percentageTotalDelay, totalDelay)

In [3]:
# Function for filtering (part II)
def filter_II(df_sent, df_received, a):
    # Sent
    # 3) Filter messages with macros 1001, 1002 and 1003
    df_sent = df_sent.loc[(df_sent['HM_NUM_MSG']==1001)|(df_sent['HM_NUM_MSG']==1002)|(df_sent['HM_NUM_MSG']==1003)]
    # 4) Filter messages that took longer than 5 minutes
    df_sent = df_sent.loc[(df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']) >= a]
    # 5) Order the dataframe by HM_ID_HM
    df_sent.sort_values(by=['HM_ID_HM'])

    #Received
    # 5) Filter messages with macros 2006, 2011, 2012, 2015, 2017, 2020, 2021 and 2029
    df_received = df_received.loc[(df_received['HM_NUM_MSG']==2006)|(df_received['HM_NUM_MSG']==2011)|(df_received['HM_NUM_MSG']==2012)|(df_received['HM_NUM_MSG']==2015)|(df_received['HM_NUM_MSG']==2017)|(df_received['HM_NUM_MSG']==2020)|(df_received['HM_NUM_MSG']==2021)|(df_received['HM_NUM_MSG']==2029)]
    # 6) Filter messages that took longer than 5 minutes
    df_received = df_received.loc[(df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']) >= a]
    # 7) Order the dataframe by HM_ID_HM
    df_received.sort_values(by=['HM_ID_HM'])
    # 8) Remove messages that don't contain data
    df_received = df_received.loc[df_received['HM_TXT_MSG'].notnull()]
    
    return(df_sent, df_received)

In [8]:
# Function to return total of delays per SB and total delivery time per SB for sent, received and total(sent + received) messages
def sb_calc(df_sb, df_sent, df_received, totalSentMessageDelay, totalReceivedMessageDelay):
    
    # Sent
    for i in range(0,len(df_sb['NAME']),1):
        for j in range(0,totalSentMessageDelay,1):
            if df_sent.iloc[j,3] == 1002 and df_sb.iloc[i,1] in df_sent.iloc[j,6]:
                df_sb.iloc[i,4] += 1
                df_sb.iloc[i,7] += df_sent.iloc[j,11]

            if df_sent.iloc[j,3] != 1002 and df_sb.iloc[i,2] in df_sent.iloc[j,6]:
                df_sb.iloc[i,4] +=1
                df_sb.iloc[i,7] += df_sent.iloc[j,11]

    # Received
    for i in range(0,len(df_sb['NAME']),1):
        for j in range(0,totalReceivedMessageDelay,1):
            if df_sb.iloc[i,1] in df_received.iloc[j,6]:
                df_sb.iloc[i,5] += 1
                df_sb.iloc[i,8] += df_received.iloc[j,11]

            if df_sb.iloc[i,2] in df_received.iloc[j,6]:
                df_sb.iloc[i,5] += 1
                df_sb.iloc[i,8] += df_received.iloc[j,11]

            if df_sb.iloc[i,3] in df_received.iloc[j,6]:
                df_sb.iloc[i,5] += 1
                df_sb.iloc[i,8] += df_received.iloc[j,11]

    # !! There may be a slight difference between totalReceivedMessageDelay and the sum of df_sb['COUNTS'].
    # This is because the script does not consider messages on switches 
    # There are also some messages that have 'Canton:' but don't contain an actual location !!

    # Total
    for i in range(0, len(df_sb['NAME']),1):
        df_sb.iloc[i,6] = df_sb.iloc[i,4] + df_sb.iloc[i,5]
        df_sb.iloc[i,9] = df_sb.iloc[i,7] + df_sb.iloc[i,8]

    # Fix for Received Messages Count
    x=0
    for i in range(0,len(df_sb['NAME']),1):
        x = x + df_sb.iloc[i,5]

    # A new value is returned to 'totalReceivedMessageDelay' to correctly count the % column.
    # Please refer to the difference in the comments of the previous section. (Between '!!...!!')
    totalReceivedMessageDelay = int(x)
    
    return (df_sb,totalReceivedMessageDelay)

In [1]:
# Function to calculate the average delay per SB
def aver_delay_sb(df_sb):
    # Packages
    from datetime import timedelta
    
    # Sent
    for i in range(0,len(df_sb['NAME']),1):
        if df_sb.iloc[i,4] != 0:
            df_sb.iloc[i,10] = df_sb.iloc[i,7]/df_sb.iloc[i,4]
            df_sb.iloc[i,10] = round(df_sb.iloc[i,10],0)    # Have to round the values to Integer to avoid having milliseconds
            df_sb.iloc[i,10] = str(timedelta(seconds=df_sb.iloc[i,10])) # Converting to String time format
            df_sb.iloc[i,7] = str(timedelta(seconds=df_sb.iloc[i,7]))
        else:
            df_sb.iloc[i,10] = 0      
            df_sb.iloc[i,10] = str(timedelta(seconds=df_sb.iloc[i,10]))
            df_sb.iloc[i,7] = str(timedelta(seconds=df_sb.iloc[i,7]))

    # Received
    for i in range(0,len(df_sb['NAME']),1):
        if df_sb.iloc[i,5] != 0:
            df_sb.iloc[i,11] = df_sb.iloc[i,8]/df_sb.iloc[i,5]
            df_sb.iloc[i,11] = round(df_sb.iloc[i,11],0)    # Have to round the values to Integer to avoid having milliseconds
            df_sb.iloc[i,11] = str(timedelta(seconds=df_sb.iloc[i,11])) # Converting to String time format
            df_sb.iloc[i,8] = str(timedelta(seconds=df_sb.iloc[i,8]))
        else:
            df_sb.iloc[i,11] = 0      
            df_sb.iloc[i,11] = str(timedelta(seconds=df_sb.iloc[i,11]))
            df_sb.iloc[i,8] = str(timedelta(seconds=df_sb.iloc[i,8]))

    # Total
    for i in range(0,len(df_sb['NAME']),1):
        if df_sb.iloc[i,6] != 0:
            df_sb.iloc[i,12] = df_sb.iloc[i,9]/df_sb.iloc[i,6]
            df_sb.iloc[i,12] = round(df_sb.iloc[i,12],0)    # Have to round the values to Integer to avoid having milliseconds
            df_sb.iloc[i,12] = str(timedelta(seconds=df_sb.iloc[i,12])) # Converting to String time format
            df_sb.iloc[i,9] = str(timedelta(seconds=df_sb.iloc[i,9]))
        else:
            df_sb.iloc[i,12] = 0      
            df_sb.iloc[i,12] = str(timedelta(seconds=df_sb.iloc[i,12]))
            df_sb.iloc[i,9] = str(timedelta(seconds=df_sb.iloc[i,9]))
            
    return(df_sb)

In [ ]:
# Function to calculate the % of total delays per Section Block
def perc_sb(df_sb, totalSentMessageDelay, totalReceivedMessageDelay):
    # Sent
    for i in range(0,len(df_sb['NAME']),1):
        df_sb.iloc[i,13] = round((df_sb.iloc[i,4]/totalSentMessageDelay)*100,2)
        df_sb.iloc[i,13] = str(df_sb.iloc[i,13])+'%'

    # Received
    for i in range(0,len(df_sb['NAME']),1):
        df_sb.iloc[i,14] = round((df_sb.iloc[i,5]/totalReceivedMessageDelay)*100,2)
        df_sb.iloc[i,14] = str(df_sb.iloc[i,14])+'%'

    # Total
    for i in range(0,len(df_sb['NAME']),1):
        df_sb.iloc[i,15] = round((df_sb.iloc[i,6]/(totalSentMessageDelay+totalReceivedMessageDelay))*100,2)
        df_sb.iloc[i,15] = str(df_sb.iloc[i,15])+'%'
        
    return (df_sb)

In [ ]:
# Function to create Density graph 
def disp_all_graph(df_sb, month, year):
    #Packages
    import matplotlib.pyplot as plt
    
    plt.figure(1)
    plt.subplot(2,2,1)
    plt.scatter(df_sb.iloc[:,0],df_sb.iloc[:,4], color='cornflowerblue', marker='.')
    plt.xticks([])
    for i in range(0,max(df_sb.iloc[:,4]),2):
        plt.axhline(y=i, color='black', linestyle='--',linewidth=0.5)
    plt.title("Sent Messages " + month + '-' + year)
    plt.subplot(2,2,2)
    plt.scatter(df_sb.iloc[:,0],df_sb.iloc[:,5], color='tab:red', marker='.')
    plt.xticks([])
    for i in range(0,max(df_sb.iloc[:,5]),10):
        plt.axhline(y=i, color='black', linestyle='--',linewidth=0.5)
    plt.title("Received Messages " + month + '-' + year)
    plt.subplot(2,2,3)
    plt.scatter(df_sb.iloc[:,0],df_sb.iloc[:,6], color='olivedrab', marker='.')
    plt.xticks([])
    for i in range(0,max(df_sb.iloc[:,6]),10):
        plt.axhline(y=i, color='black', linestyle='--',linewidth=0.5)
    plt.title("All Messages " + month + '-' + year) 
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/dispersion_all'+'_'+ month + '_' + year +'.png')
    plt.show()

In [ ]:
# Function to create Density graph of sent and received messages combined
def disp_sent_received_graph(df_sb, month, year):
    #Packages
    import matplotlib.pyplot as plt
    
    plt.scatter(df_sb.iloc[:,0],df_sb.iloc[:,4], color='cornflowerblue', marker='.', label='Sent')
    plt.xticks([])
    for i in range(0,max(df_sb.iloc[:,4]),10):
        plt.axhline(y=i, color='black', linestyle='--',linewidth=0.5)

    plt.scatter(df_sb.iloc[:,0],df_sb.iloc[:,5], color='tab:red', marker='.', label='Received')
    plt.xticks([])
    for i in range(0,max(df_sb.iloc[:,5]),10):
        plt.axhline(y=i, color='black', linestyle='--',linewidth=0.5)
    plt.legend(loc='best')
    plt.title("Delivery of >5 min. per Section Block " + month + '-' + year)
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/dispersion_sent_received'+'_'+ month + '_' + year +'.png')
    plt.show()

In [ ]:
# Function to create Density graph of total messages
def disp_total_graph(df_sb, month, year):
    #Packages
    import matplotlib.pyplot as plt
    
    plt.scatter(df_sb.iloc[:,0],df_sb.iloc[:,6], color='olivedrab', marker='.')
    plt.xticks([])
    for i in range(0,max(df_sb.iloc[:,6]),10):
        plt.axhline(y=i, color='black', linestyle='--',linewidth=0.5)
    plt.title("Delivery of >5 min. per Section Block " + month + '-' + year)
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/dispersion_total'+'_'+ month + '_' + year +'.png')
    plt.show()

In [1]:
# Function to select TOP 10 for barplot graph
def order_barplot(df_sb):
    # Ordering values
    ordered_Sent = df_sb.sort_values(by=['COUNTS_SENT'], ascending=False)
    ordered_Received = df_sb.sort_values(by=['COUNTS_RECEIVED'], ascending=False)
    ordered_Total = df_sb.sort_values(by=['COUNTS_TOTAL'], ascending=False)

    top10_Sent = ordered_Sent.head(n=10)
    top10_Received = ordered_Received.head(n=10)
    top10_Total = ordered_Total.head(n=10)

    top10_Sent = top10_Sent.sort_values(by=['COUNTS_SENT'], ascending=True)
    top10_Received = top10_Received.sort_values(by=['COUNTS_RECEIVED'], ascending=True)
    top10_Total = top10_Total.sort_values(by=['COUNTS_TOTAL'], ascending=True)
    
    return(top10_Sent, top10_Received, top10_Total)

In [ ]:
# Function to create horizontal barplot for sent, received and total SBs combined

def barplot_all(df_sb, top10_Sent, top10_Received, top10_Total, month, year):
    # Packages
    import matplotlib.pyplot as plt
    
    plt.figure(1)
    plt.subplot(2,2,1)
    plt.barh(top10_Sent.iloc[:,0],top10_Sent.iloc[:,4], color ='cornflowerblue',align='center')
    plt.xlim(0,max(df_sb.iloc[:,4])+10)
    plt.xlabel("Number of messages") 
    plt.ylabel("Top 10 Section Blocks")
    plt.title("Sent Messages " + month + '-' + year) 
    # Adding percentage values of '%' column to the bars
    #for index, value in enumerate(top10_Sent.iloc[:,4]):
    #    plt.text(value, index, str(top10_Sent.iloc[index,13]))
    plt.subplot(2,2,2)
    plt.barh(top10_Received.iloc[:,0],top10_Received.iloc[:,5], color ='tab:red',align='center')
    plt.xlim(0,max(df_sb.iloc[:,5])+10)
    plt.title("Received Messages" + month + '-' + year) 
    plt.subplot(2,2,3)
    plt.barh(top10_Total.iloc[:,0],top10_Total.iloc[:,6], color ='olivedrab',align='center')
    plt.xlim(0,max(df_sb.iloc[:,6])+10)
    plt.title("All Messages" + month + '-' + year) 
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/bar_all'+'_'+ month + '_' + year +'.png')
    plt.show()

In [ ]:
# Function to create horizontal barplot for sent SBs
def barplot_sent(df_sb, top10_Sent, month, year):
    # Packages
    import matplotlib.pyplot as plt
    
    plt.barh(top10_Sent.iloc[:,0],top10_Sent.iloc[:,4], color ='cornflowerblue',align='center')
    plt.xlim(0,max(df_sb.iloc[:,4])+10)
    plt.xlabel("Number of messages") 
    plt.ylabel("Section Blocks")
    plt.title("Sent messages with >5min. delivery time " + month + '-' + year) 
    # Adding percentage values of '%' column to the bars
    for index, value in enumerate(top10_Sent.iloc[:,4]):
        plt.text(value, index, str(top10_Sent.iloc[index,13]))
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/bar_sent'+'_'+ month + '_' + year +'.png')
    plt.show()

In [2]:
# Function to create horizontal barplot for received SBs
def barplot_received(df_sb, top10_Received, month, year):
    # Packages
    import matplotlib.pyplot as plt
    
    plt.barh(top10_Received.iloc[:,0],top10_Received.iloc[:,5], color ='tab:red',align='center')
    plt.xlim(0,max(df_sb.iloc[:,5])+10)
    plt.xlabel("Number of messages") 
    plt.ylabel("Section Blocks")
    plt.title("Received messages with >5min. delivery time " + month + '-' + year) 
    # Adding percentage values of '%' column to the bars
    for index, value in enumerate(top10_Received.iloc[:,5]):
        plt.text(value, index, str(top10_Received.iloc[index,14]))
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/bar_received'+'_'+ month + '_' + year +'.png')
    plt.show()

SyntaxError: invalid syntax (<ipython-input-2-25c6c40a2a20>, line 2)

In [ ]:
# Function to create horizontal barplot for total SBs (sent + received)
def barplot_total(df_sb, top10_Total, month, year):
    # Packages
    import matplotlib.pyplot as plt
    
    plt.barh(top10_Total.iloc[:,0],top10_Total.iloc[:,6], color ='olivedrab',align='center')
    plt.xlim(0,max(df_sb.iloc[:,6])+10)
    plt.xlabel("Number of messages") 
    plt.ylabel("Section Blocks")
    plt.title("Total messages with >5min. delivery time " + month + '-' + year) 
    # Adding percentage values of '%' column to the bars
    for index, value in enumerate(top10_Total.iloc[:,6]):
        plt.text(value, index, str(top10_Total.iloc[index,15]))
    plt.savefig('D:/app/kenne/product/18.0.0/admin/XE/images/bar_total'+'_'+ month + '_' + year +'.png')
    plt.show()

In [4]:
# Function to prepare images for database insertion
def var_prep(month, year):

    density = 'density_'+month+'_'+year+'.png'
    dispersion_all = 'dispersion_all_'+month+'_'+year+'.png'
    dispersion_sent_received = 'dispersion_sent_received_'+month+'_'+year+'.png'
    dispersion_total = 'dispersion_total_'+month+'_'+year+'.png'
    bar_all = 'bar_all_'+month+'_'+year+'.png'
    bar_sent = 'bar_sent_'+month+'_'+year+'.png'
    bar_received = 'bar_received_'+month+'_'+year+'.png'
    bar_total = 'bar_total_'+month+'_'+year+'.png'
    #time_series = 'time_series_'+month+'_'+year+'.png' 
    #time_series_decompose = 'time_series_decompose'+month+'_'+year+'.png' 

    month = int(month)
    year = int(year)
    
    return(density, dispersion_all, dispersion_sent_received, dispersion_total, bar_all, bar_sent, bar_received, bar_total, month, year)

In [ ]:
# Function for database insertion
def db_insert(month, year, density, dispersion_all, dispersion_sent_received, dispersion_total, bar_all, bar_sent, bar_received, bar_total, totalSentMessages, averageSentDelivery, percentageSentDelay, totalSentDelay, totalSentMessageDelay, averageSentDeliveryDelay, totalReceivedMessages, averageReceivedDelivery, percentageReceivedDelay, totalReceivedDelay, totalReceivedMessageDelay, averageReceivedDeliveryDelay,totalMessages, averageTotalDelivery, percentageTotalDelay, totalDelay, averageDeliveryDelay):
    # Packages
    import cx_Oracle
    import pandas as pd
    
    con = cx_Oracle.connect('DATA_ANALYSIS/art@localhost/XE')
    cur = con.cursor()

    cur.execute("insert into SENT (ID_SENT, MONTH, YEAR, TOTALSENTMESSAGES, AVERAGESENTDELIVERY, PERCENTAGESENTDELAY, TOTALSENTDELAY, TOTALSENTMESSAGEDELAY, averageSentDeliveryDelay, regtimestamp) values (SENT_ID_SEQ.NEXTVAL, :month, :year, :totalSentMessages, :averageSentDelivery, :percentageSentDelay, :totalSentDelay, :totalSentMessageDelay, :averageSentDeliveryDelay, sysdate)", [month, year, totalSentMessages, averageSentDelivery, percentageSentDelay, totalSentDelay, totalSentMessageDelay, averageSentDeliveryDelay])
    cur.execute("insert into RECEIVED (ID_RECEIVED, MONTH, YEAR, TOTALRECEIVEDMESSAGES, AVERAGERECEIVEDDELIVERY, PERCENTAGERECEIVEDDELAY, TOTALRECEIVEDDELAY, TOTALRECEIVEDMESSAGEDELAY, averageReceivedDeliveryDelay, regtimestamp) values (RECEIVED_ID_SEQ.NEXTVAL, :month, :year, :totalReceivedMessages, :averageReceivedDelivery, :percentageReceivedDelay, :totalReceivedDelay, :totalReceivedMessageDelay, :averageReceivedDeliveryDelay, sysdate)", [month, year, totalReceivedMessages, averageReceivedDelivery, percentageReceivedDelay, totalReceivedDelay, totalReceivedMessageDelay, averageReceivedDeliveryDelay])
    cur.execute("insert into TOTAL (ID_TOTAL, MONTH, YEAR, TOTALMESSAGES, AVERAGETOTALDELIVERY, PERCENTAGETOTALDELAY, TOTALDELAY, averageDeliveryDelay, regtimestamp) values (TOTAL_ID_SEQ.NEXTVAL, :month, :year, :totalMessages, :averageTotalDelivery, :percentageTotalDelay, :totalDelay, :averageDeliveryDelay, sysdate)", [month, year, totalMessages, averageTotalDelivery, percentageTotalDelay, totalDelay, averageDeliveryDelay])
    cur.execute("insert into GRAPHS (ID_GRAPHS, MONTH, YEAR, regtimestamp) values (GRAPHS_ID_SEQ.NEXTVAL, :month, :year, sysdate)", [month, year])
    cur.callproc("graphs_pr",[month, year, density, dispersion_all, dispersion_sent_received, dispersion_total, bar_all, bar_sent, bar_received, bar_total])
    con.commit()


    cur.close()
    con.close() 

In [ ]:
##### TO ADD!!
 # the dataframe df_sb should also be inserted into the database!!
 # user input for month and year variables
 # add timeseries
 # add delays per locomotives